In [3]:
import os
from datetime import datetime, timedelta
import datetime
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
DATADIR = "D:\My Computer\Local H\Skoltech Docs\Courses\HackLab\hackathon_data" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"
df = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

# Feature Generation
## We generated the following features:
1. Holiday check

In [7]:
dates = pd.to_datetime(df['transaction_dttm'], format = '%Y-%m-%d')


def weekend(date):
    weekno = datetime.datetime(date.year,date.month,date.day).weekday()
    if weekno<5:
        return 0
    else:
        return 1

dates = dates.apply(weekend)

df["isweekend"] = dates

In [8]:
df

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category,isweekend
0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры,0
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд,0
2,24789,23517,2,2019-01-01,Покупка,701.44,830014.0,291.0,NaN,Супермаркеты,0
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт,0
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты,0
...,...,...,...,...,...,...,...,...,...,...,...
11987612,44990,111641,1,2019-12-31,Покупка,121.00,1940970.0,330.0,675.0,Фаст Фуд,0
11987613,53023,57593,2,2019-12-31,Покупка,102.29,1198564.0,286.0,NaN,Супермаркеты,0
11987614,78716,99133,1,2019-12-31,Покупка,175.00,2688766.0,229.0,901.0,Транспорт,0
11987615,73104,91039,2,2019-12-31,Покупка,185.99,1965313.0,286.0,878.0,Супермаркеты,0


In [9]:
dates = pd.to_datetime(df['transaction_dttm'], format = '%Y-%m-%d')
def get_month(date):
    return date.month

def get_day(date):
    return date.day

months = dates.apply(get_month)
days = dates.apply(get_day)
df_f = df.dropna()
df_f.to_csv('transaction_dataset_modified.csv')

In [5]:
#Load dataset
df_f = pd.read_csv('transaction_dataset_modified.csv')

In [10]:
df_f

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category,isweekend
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд,0
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт,0
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты,0
5,51074,54639,2,2019-01-01,Покупка,366.60,146084.0,286.0,878.0,Супермаркеты,0
6,1534,5610,2,2019-01-01,Покупка,101.85,166649.0,286.0,341.0,Супермаркеты,0
...,...,...,...,...,...,...,...,...,...,...,...
11987609,53765,109370,1,2019-12-31,Покупка,3065.00,954580.0,313.0,261.0,Одежда/Обувь,0
11987612,44990,111641,1,2019-12-31,Покупка,121.00,1940970.0,330.0,675.0,Фаст Фуд,0
11987614,78716,99133,1,2019-12-31,Покупка,175.00,2688766.0,229.0,901.0,Транспорт,0
11987615,73104,91039,2,2019-12-31,Покупка,185.99,1965313.0,286.0,878.0,Супермаркеты,0


# Machine Learning Models

In [11]:
# Input/Output
X = df_f[['party_rk','account_rk','financial_account_type_cd','isweekend']]
X['Month'] = months
X['Day'] = days
Y2 = df_f[['merchant_type']]

C:\Users\basel\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\basel\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.3, random_state=42)

In [14]:
#Classification model (K Neighbors Classifier)

KN_clf = KNeighborsClassifier(n_neighbors = 50 ,n_jobs=-1)
model_fit = KN_clf.fit(X_train,y_train)

C:\Users\basel\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
y_pred = model_fit.predict(X_test)

In [17]:
accuracy_score(y_test, y_pred)

0.46545449256335486

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [17]:
#Stochastic Gradient Descent

clf_sgd = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf_sgd.fit(X_train, y_train)

C:\Users\basel\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgdclassifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=None,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [20]:
y_pred_sgd = clf_sgd.predict(X_test)

In [21]:
accuracy_score(y_test, y_pred_sgd)

0.20101929372898472

In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
#Naive Bayes

NB_classifier = GaussianNB()
NB_classifier.fit(X_train, y_train)
y_pred_NB = NB_classifier.predict(X_test)

C:\Users\basel\anaconda3\lib\site-packages\sklearn\naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
accuracy_score(y_test, y_pred_NB)

0.4082139214097848